In [1]:
#import packages
import pandas as pd
import requests
import os as os
import json 
import tweepy
from timeit import default_timer as timer
import ast

# Gathring #

In [168]:
df_csv=pd.read_csv("twitter-archive-enhanced.csv")

In [3]:
imgpre_url="https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

In [4]:
imgpre_filename=imgpre_url.split("/")[-1]

In [5]:
response=requests.get(imgpre_url)

In [6]:
with open(imgpre_filename,mode="wb") as file:
    file.write(response.content)

In [7]:
df_img=pd.read_csv(imgpre_filename,sep="\t")

In [8]:
credintials=pd.read_json("tweeter_api_token.json")

In [224]:
#credintials.head()

In [10]:
auth=tweepy.OAuthHandler(credintials.tokens.APIkey,credintials.tokens.APISecretkey)
auth.set_access_token(credintials.tokens.AccessToken,credintials.tokens.AccessTokenSecret)

In [11]:
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

start = timer()
    failer={}
    Readinglog=""
    n=0
    with open ("tweet_json.txt",mode="w") as file:
        for tweetid in df_csv.tweet_id:
            n=n+1
            Readinglog+=(str(n)+" "+str(tweetid)+" ")
            print ( n," : ",timer() )
            try:
                temp=api.get_status(tweetid, tweet_mode='extended')._json
                Readinglog+="Success "
                json.dump(temp,file)            
                file.writelines("\n")
            except tweepy.TweepError as e:
                Readinglog+="Faild "
                failer[tweetid]=e
                pass
            #print(temp["favorite"])#favorite
            Readinglog+="\n"
            if n == 10000:
                break
    end = timer()
    print ( end - start)
    
    start = timer()
    failer={}
    Readinglog1=""
    n=0
    with open ("tweet_json1.txt",mode="w") as file:
        for tweetid in df_csv.tweet_id:
            n=n+1
            Readinglog1+=(str(n)+" "+str(tweetid)+" ")
            print ( n," : ",timer() )
            try:
                temp=api.get_status(tweetid, tweet_mode='extended')._json
                Readinglog1+="Success "
                json.dump(temp,file)            
                file.writelines("\n")
            except tweepy.TweepError as e:
                Readinglog1+="Faild "
                failer[tweetid]=e
                pass
            #print(temp["favorite"])#favorite
            Readinglog1+="\n"
            if n == 10:
                break

    end = timer()
    print ( end - start)

In [14]:
#https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
with open ("tweet_json1.txt",mode="r") as file:
    lines=file.readlines()

In [15]:
json.loads(lines[3])

{'created_at': 'Sun Jul 30 15:58:51 +0000 2017',
 'id': 891689557279858688,
 'id_str': '891689557279858688',
 'full_text': 'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ',
 'truncated': False,
 'display_text_range': [0, 79],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 891689552724799489,
    'id_str': '891689552724799489',
    'indices': [80, 103],
    'media_url': 'http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg',
    'url': 'https://t.co/tD36da7qLQ',
    'display_url': 'pic.twitter.com/tD36da7qLQ',
    'expanded_url': 'https://twitter.com/dog_rates/status/891689557279858688/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'small': {'w': 510, 'h': 680, 'resize': 'fit'},
     'medium': {'w': 901, 'h': 1200, 'resize': 'fit'},
     'large': {'w': 1201,

In [16]:
with open ("tweet_json.txt",mode="r") as file:
    lines=file.readlines()

In [61]:
df_list=[]
for line in lines:
    line=json.loads(line)
    df_list.append({
        "tweet_id":line["id"],
        "favorite_count":line["favorite_count"],
        "retweet_count":line["retweet_count"],
           })

In [62]:
df_api=pd.DataFrame(df_list)

In [63]:
df_api.head()

,favorite_count,retweet_count,tweet_id
0,37731,8221,892420643555336193
1,32404,6077,892177421306343426
2,24401,4022,891815181378084864
3,41040,8376,891689557279858688
4,39238,9079,891327558926688256


In [128]:
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [129]:
df_csv.head(50)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,name,doggo,floofer,pupper,puppo,rating
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,nan,nan,NaN,https://twitter.com/dog_rates/status/892420643...,Phineas,None,None,None,None,1.3
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,nan,nan,NaN,https://twitter.com/dog_rates/status/892177421...,Tilly,None,None,None,None,1.3
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,nan,nan,NaN,https://twitter.com/dog_rates/status/891815181...,Archie,None,None,None,None,1.2
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,nan,nan,NaN,https://twitter.com/dog_rates/status/891689557...,Darla,None,None,None,None,1.3
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,nan,nan,NaN,https://twitter.com/dog_rates/status/891327558...,Franklin,None,None,None,None,1.2
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,nan,nan,NaN,https://twitter.com/dog_rates/status/891087950...,None,None,None,None,None,1.3
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,nan,nan,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",Jax,None,None,None,None,1.3
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,nan,nan,NaN,https://twitter.com/dog_rates/status/890729181...,None,None,None,None,None,1.3
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,nan,nan,NaN,https://twitter.com/dog_rates/status/890609185...,Zoey,None,None,None,None,1.3
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,nan,nan,NaN,https://twitter.com/dog_rates/status/890240255...,Cassie,doggo,None,None,None,1.4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

 # Assest  # 
## Quality: ## 
 
 ### df_csv ###
 1 - time stamp need to be changed to datatime instead of string<br/>
 2 - dummy varialbs ( dog type ) need to be cleaned into int ( boolean)<br/>
 3 - in_reply_to_status_id should be changed to string type <br/>
 4 - retweeted_status_user_id  should be changed to string type <br/>
 5 - rating_numerator ,rating_denominator  changing into one float variable<br/>
 <br/>
 ### df_img: ###
 changing id's into string <br/>
 <br/>
 #### df_api ####
 <br/>
 changing id's into string
 
 

In [ ]:
df_csv.head()

In [39]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [23]:
df_csv.doggo.unique()

array(['None', 'doggo'], dtype=object)

In [24]:
df_csv.floofer.unique()

array(['None', 'floofer'], dtype=object)

In [25]:
df_csv.puppo.unique()

array(['None', 'puppo'], dtype=object)

In [26]:
df_csv.pupper.unique()

array(['None', 'pupper'], dtype=object)

In [32]:
df_csv[df_csv.expanded_urls.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 30 to 2298
Data columns (total 17 columns):
tweet_id                      59 non-null int64
in_reply_to_status_id         55 non-null float64
in_reply_to_user_id           55 non-null float64
timestamp                     59 non-null object
source                        59 non-null object
text                          59 non-null object
retweeted_status_id           1 non-null float64
retweeted_status_user_id      1 non-null float64
retweeted_status_timestamp    1 non-null object
expanded_urls                 0 non-null object
rating_numerator              59 non-null int64
rating_denominator            59 non-null int64
name                          59 non-null object
doggo                         59 non-null object
floofer                       59 non-null object
pupper                        59 non-null object
puppo                         59 non-null object
dtypes: float64(4), int64(3), object(10)
memory usage: 8.3+ KB


In [33]:
df_csv[df_csv.expanded_urls.isna()].head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None


In [38]:
df_csv[df_csv.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [43]:
df_img.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [44]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [47]:
df_img.p3.unique()

array(['Shetland_sheepdog', 'Rhodesian_ridgeback', 'bloodhound',
       'miniature_pinscher', 'Doberman', 'Greater_Swiss_Mountain_dog',
       'terrapin', 'fur_coat', 'golden_retriever',
       'soft-coated_wheaten_terrier', 'Labrador_retriever', 'Pekinese',
       'Ibizan_hound', 'French_bulldog', 'malinois', 'Dandie_Dinmont',
       'borzoi', 'partridge', 'bookcase', 'basenji', 'miniature_poodle',
       'great_grey_owl', 'groenendael', 'Eskimo_dog', 'hamster', 'briard',
       'papillon', 'flat-coated_retriever', 'gar', 'Chihuahua',
       'Shih-Tzu', 'Pomeranian', 'dingo', 'power_drill', 'Saluki',
       'Great_Pyrenees', 'West_Highland_white_terrier', 'collie',
       'toy_poodle', 'vizsla', 'acorn', 'giant_schnauzer', 'teddy',
       'common_iguana', 'wig', 'water_buffalo', 'coyote', 'seat_belt',
       'kelpie', 'space_heater', 'Brabancon_griffon', 'standard_poodle',
       'beagle', 'Irish_water_spaniel', 'bluetick', 'Weimaraner',
       'Chesapeake_Bay_retriever', 'toilet_tiss

In [50]:
df_img.img_num.unique()

array([1, 4, 2, 3], dtype=int64)

In [56]:
df_img[df_img.img_num==4].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 144 to 2040
Data columns (total 12 columns):
tweet_id    31 non-null int64
jpg_url     31 non-null object
img_num     31 non-null int64
p1          31 non-null object
p1_conf     31 non-null float64
p1_dog      31 non-null bool
p2          31 non-null object
p2_conf     31 non-null float64
p2_dog      31 non-null bool
p3          31 non-null object
p3_conf     31 non-null float64
p3_dog      31 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 2.5+ KB


In [57]:
df_img[df_img.img_num==4].head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
144,668623201287675904,https://pbs.twimg.com/media/CUdtP1xUYAIeBnE.jpg,4,Chihuahua,0.708163,True,Pomeranian,0.091372,True,titi,0.067325,False
779,689905486972461056,https://pbs.twimg.com/media/CZMJYCRVAAE35Wk.jpg,4,Pomeranian,0.943331,True,Shetland_sheepdog,0.023675,True,chow,0.007165,True
1024,710588934686908417,https://pbs.twimg.com/media/CdyE2x1W8AAe0TG.jpg,4,Pembroke,0.982004,True,Cardigan,0.008943,True,malamute,0.007550,True
1161,734787690684657664,https://pbs.twimg.com/media/CjJ9gQ1WgAAXQtJ.jpg,4,golden_retriever,0.883991,True,chow,0.023542,True,Labrador_retriever,0.016056,True
1286,750868782890057730,https://pbs.twimg.com/media/CmufLLsXYAAsU0r.jpg,4,toy_poodle,0.912648,True,miniature_poodle,0.035059,True,seat_belt,0.026376,False


In [48]:
 df_api.head()

,favorite_count,id,retweet_count
0,37731,892420643555336193,8221
1,32404,892177421306343426,6077
2,24401,891815181378084864,4022
3,41040,891689557279858688,8376
4,39238,891327558926688256,9079


In [49]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 3 columns):
favorite_count    2339 non-null int64
id                2339 non-null int64
retweet_count     2339 non-null int64
dtypes: int64(3)
memory usage: 54.9 KB


  # Assest  # 

 ## tideness ##
 
1 -  df_api , df_csv  represent the same obeservations it would better to be merged together
 but we should be careful about the nullable and deference count between the raw<br/>
2 - merged , df_img represent the same obeservations it would better to be merged together
 but we should be careful about the nullable and deference count between the raw

# Cleaning #
## Quality ##

### df_csv ###
1- chaging time into datatime type

In [169]:
df_csv["timestamp"]=pd.to_datetime(df_csv.timestamp)

In [170]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64[ns](1

### df_csv ###
2- chaging retweeted_status_id and other id's into string type


In [199]:
df_csv["tweet_id"]=df_csv["tweet_id"].astype(str)
df_csv["retweeted_status_id"]=df_csv["retweeted_status_id"].astype(str)
df_csv["retweeted_status_user_id"]=df_csv["retweeted_status_user_id"].astype(str)
df_csv["in_reply_to_status_id"]=df_csv["in_reply_to_status_id"].astype(str)
df_csv["in_reply_to_user_id"]=df_csv["in_reply_to_user_id"].astype(str)


### df_csv ###
3- changing rating into one column and droping the numerator and denominator note ( denominator = 0 ) should be droped


In [172]:
df_csv=df_csv[df_csv["rating_denominator"]!=0]

In [173]:
df_csv["rating"]=df_csv.rating_numerator / df_csv.rating_denominator

In [174]:
df_csv.drop(["rating_denominator","rating_numerator"],axis=1,inplace=True)

### df_csv ###
3- changing the dummy variabls into int type int( not boolean to get easy the discretive data even its int between 1 and 0)


In [175]:
df_csv["doggo"]=(df_csv["doggo"]=="doggo").astype(int)

In [176]:
df_csv["floofer"]=(df_csv["floofer"]=="floofer").astype(int)
df_csv["pupper"]=(df_csv["pupper"]=="pupper").astype(int)
df_csv["puppo"]=(df_csv["puppo"]=="puppo").astype(int)


In [195]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2355 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2355 non-null int64
in_reply_to_status_id         77 non-null float64
in_reply_to_user_id           77 non-null float64
timestamp                     2355 non-null datetime64[ns]
source                        2355 non-null object
text                          2355 non-null object
retweeted_status_id           2355 non-null object
retweeted_status_user_id      2355 non-null object
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
name                          2355 non-null object
doggo                         2355 non-null int32
floofer                       2355 non-null int32
pupper                        2355 non-null int32
puppo                         2355 non-null int32
rating                        2355 non-null float64
none                          2355 non-null int32
dtypes: datetime64[ns](1),

In [188]:
df_csv["none"]= (df_csv["doggo"]+df_csv["floofer"]+df_csv["pupper"]+df_csv["puppo"])==0

In [189]:
df_csv["none"]=df_csv["none"].astype(int)

In [200]:
df_csv.describe()

,doggo,floofer,pupper,puppo,rating,none
count,2355.000000,2355.000000,2355.000000,2355.000000,2355.000000,2355.000000
mean,0.041189,0.004246,0.109130,0.012739,1.222032,0.838641
std,0.198769,0.065039,0.311868,0.112169,4.083485,0.367940
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,1.100000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,1.200000,1.000000
max,1.000000,1.000000,1.000000,1.000000,177.600000,1.000000


### df_img  ###
1- chaging id into string

In [206]:
df_img["tweet_id"]=df_img["tweet_id"].astype(str)

In [207]:
df_img.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(1), object(5)
memory usage: 152.1+ KB


In [208]:
df_img.describe()

,img_num,p1_conf,p2_conf,p3_conf
count,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,1.203855,0.594548,1.345886e-01,6.032417e-02
std,0.561875,0.271174,1.006657e-01,5.090593e-02
min,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,1.000000,0.843855,1.955655e-01,9.180755e-02
max,4.000000,1.000000,4.880140e-01,2.734190e-01


### df_api  ###
1- chaging id into string

In [205]:
df_api["tweet_id"]=df_api["tweet_id"].astype(str)

In [209]:
df_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 3 columns):
favorite_count    2339 non-null int64
retweet_count     2339 non-null int64
tweet_id          2339 non-null object
dtypes: int64(2), object(1)
memory usage: 54.9+ KB


In [210]:
df_api.describe()

,favorite_count,retweet_count
count,2339.000000,2339.000000
mean,7887.823001,2899.079949
std,12224.011478,4888.510848
min,0.000000,0.000000
25%,1366.500000,583.000000
50%,3431.000000,1353.000000
75%,9658.500000,3379.000000
max,162830.000000,82862.000000


# Cleaning #
## Tidiness ##

 ## df_api ,  df_csv ## 
represent the same obeservations it would better to be merged together but we should be careful about the nullable and deference count between the raw

In [212]:
merged_1=df_csv.merge(df_api,how="inner",on="tweet_id")

In [214]:
merged_1.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,name,doggo,floofer,pupper,puppo,rating,none,favorite_count,retweet_count
0,892420643555336193,nan,nan,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,nan,nan,NaN,https://twitter.com/dog_rates/status/892420643...,Phineas,0,0,0,0,1.3,1,37731,8221
1,892177421306343426,nan,nan,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,nan,nan,NaN,https://twitter.com/dog_rates/status/892177421...,Tilly,0,0,0,0,1.3,1,32404,6077
2,891815181378084864,nan,nan,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,nan,nan,NaN,https://twitter.com/dog_rates/status/891815181...,Archie,0,0,0,0,1.2,1,24401,4022
3,891689557279858688,nan,nan,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,nan,nan,NaN,https://twitter.com/dog_rates/status/891689557...,Darla,0,0,0,0,1.3,1,41040,8376
4,891327558926688256,nan,nan,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,nan,nan,NaN,https://twitter.com/dog_rates/status/891327558...,Franklin,0,0,0,0,1.2,1,39238,9079


In [215]:
merged_1.describe()

,doggo,floofer,pupper,puppo,rating,none,favorite_count,retweet_count
count,2338.000000,2338.000000,2338.000000,2338.000000,2338.000000,2338.000000,2338.000000,2338.000000
mean,0.041061,0.004277,0.109495,0.012831,1.221970,0.838323,7890.263045,2900.286997
std,0.198473,0.065274,0.312326,0.112571,4.098289,0.368233,12226.056769,4889.207987
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1365.250000,583.500000
50%,0.000000,0.000000,0.000000,0.000000,1.100000,1.000000,3436.500000,1356.500000
75%,0.000000,0.000000,0.000000,0.000000,1.200000,1.000000,9658.750000,3379.500000
max,1.000000,1.000000,1.000000,1.000000,177.600000,1.000000,162830.000000,82862.000000


In [216]:
merged_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2338 entries, 0 to 2337
Data columns (total 19 columns):
tweet_id                      2338 non-null object
in_reply_to_status_id         2338 non-null object
in_reply_to_user_id           2338 non-null object
timestamp                     2338 non-null datetime64[ns]
source                        2338 non-null object
text                          2338 non-null object
retweeted_status_id           2338 non-null object
retweeted_status_user_id      2338 non-null object
retweeted_status_timestamp    167 non-null object
expanded_urls                 2280 non-null object
name                          2338 non-null object
doggo                         2338 non-null int32
floofer                       2338 non-null int32
pupper                        2338 non-null int32
puppo                         2338 non-null int32
rating                        2338 non-null float64
none                          2338 non-null int32
favorite_count         

##    ## df_img ,  merged_1 ## 
 df_img represent the same obeservations it would better to be merged together
 but we should be careful about the nullable and deference count between the raw

In [217]:
merged_2=merged_1.merge(df_img,how="inner",on="tweet_id")

In [218]:
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2066 entries, 0 to 2065
Data columns (total 30 columns):
favorite_count                2066 non-null int64
retweet_count                 2066 non-null int64
tweet_id                      2066 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2066 non-null object
source                        2066 non-null object
text                          2066 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2066 non-null object
rating_numerator              2066 non-null int64
rating_denominator            2066 non-null int64
name                          2066 non-null object
doggo                         2066 non-null object
floofer                       2066 non-null object
pupper                        2066 no

In [220]:
merged_2.describe()

,doggo,floofer,pupper,puppo,rating,none,favorite_count,retweet_count,img_num,p1_conf,p2_conf,p3_conf
count,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2066.000000,2.066000e+03,2.066000e+03
mean,0.038722,0.003872,0.107454,0.011617,1.169595,0.844627,8332.854792,2772.818490,1.203291,0.594568,1.346716e-01,6.034151e-02
std,0.192979,0.062122,0.309765,0.107179,3.995623,0.362347,12562.080796,4830.478374,0.562172,0.271062,1.007233e-01,5.094272e-02
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1584.500000,591.000000,1.000000,0.364254,5.387868e-02,1.621080e-02
50%,0.000000,0.000000,0.000000,0.000000,1.100000,1.000000,3664.500000,1305.000000,1.000000,0.588030,1.184015e-01,4.939645e-02
75%,0.000000,0.000000,0.000000,0.000000,1.200000,1.000000,10408.500000,3199.500000,1.000000,0.843883,1.955693e-01,9.208967e-02
max,1.000000,1.000000,1.000000,1.000000,177.600000,1.000000,162830.000000,82862.000000,4.000000,1.000000,4.880140e-01,2.734190e-01


In [221]:
merged_2.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,nan,nan,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,nan,nan,NaN,https://twitter.com/dog_rates/status/892420643...,...,1,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False
1,892177421306343426,nan,nan,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,nan,nan,NaN,https://twitter.com/dog_rates/status/892177421...,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True
2,891815181378084864,nan,nan,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,nan,nan,NaN,https://twitter.com/dog_rates/status/891815181...,...,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
3,891689557279858688,nan,nan,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,nan,nan,NaN,https://twitter.com/dog_rates/status/891689557...,...,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
4,891327558926688256,nan,nan,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,nan,nan,NaN,https://twitter.com/dog_rates/status/891327558...,...,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True


# Saving Data #
## saving as csv file ##



In [223]:
merged_2.to_csv("twitter_archive_master.csv")